In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(
                database='postgresdemo',
                user='clientdemo',
                password='clientdemo4',
                host='postgresdemo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com',
                port='5432',
                sslmode='require'
            )

In [3]:
temp = conn.cursor()

In [4]:
temp.execute('SELECT * from trans')

In [5]:
trans = temp.fetchall()

In [6]:
trans[0]

('3893506',
 0,
 'com.gopaktor.subscription.v1.premium.1m',
 datetime.datetime(2016, 9, 7, 13, 59, 45, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
 'ios',
 31.28,
 0,
 0,
 '1984955')

In [ ]:
colnames = [desc[0] for desc in temp.description]

In [ ]:
colnames

In [ ]:
import yaml
with open('/home/ubuntu/MappingBuffer.yml', 'r') as f:
    doc = yaml.load(f)

In [ ]:
doc['column_map']['TRANSACTION_MASTER']

In [ ]:
import pandas as pd

In [ ]:
trans_df = pd.DataFrame(trans, columns=colnames)

In [ ]:
trans_df.head()

In [ ]:
trans_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace = True)
trans_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['revenue']:'revenue'}, inplace = True)
trans_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'}, inplace = True)
trans_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'}, inplace = True)

In [ ]:
trans_df.head()

In [ ]:
len(trans_df)

In [ ]:
cust_prod = trans_df[['cust_id', 'product_id']]

In [ ]:
cust_prod.head()

In [ ]:
values = pd.DataFrame([1]*len(cust_prod))

In [ ]:
values.rename(columns={0:'rating'}, inplace=True)

In [ ]:
cust_prod_df = pd.concat([cust_prod,values],axis=1)

In [ ]:
cust_prod_df.head()

In [ ]:
cust_prod_mat = cust_prod_df.pivot_table(index = 'cust_id', columns='product_id', values='rating')

In [ ]:
cust_prod_mat.shape

In [ ]:
cust_prod_mat.head()

In [ ]:
del cust_prod_mat.index.name
del cust_prod_mat.columns.name

In [ ]:
cust_prod_mat.reset_index(level=0, inplace=True)

In [ ]:
cust_prod_mat.head(6)

In [ ]:
cust_prod_mat.rename(columns={'index':'cust_id'}, inplace=True)

In [ ]:
cust_prod_mat = cust_prod_mat.fillna(0)

In [ ]:
cust_prod_mat.head(6)

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
data_user = cust_prod_mat.drop('cust_id', 1)

In [ ]:
data_ibs = pd.DataFrame(index=data_user.columns,columns=data_user.columns)

In [ ]:
for i in range(0,len(data_ibs.columns)) :
    for j in range(0,len(data_ibs.columns)) :
      data_ibs.ix[i,j] = 1-cosine(data_user.ix[:,i],data_user.ix[:,j])

In [ ]:
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))

In [ ]:
for i in range(0,len(data_ibs.columns)):
    data_neighbours.ix[i,:10] = data_ibs.ix[0:,i].sort_values(ascending=False)[:10].index

In [ ]:
data_neighbours.head(6)

In [ ]:
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

In [ ]:
data_sims = pd.DataFrame(index=cust_prod_mat.index,columns=cust_prod_mat.columns)

In [ ]:
data_sims.head(6)

In [ ]:
data_sims.ix[:,:1] = cust_prod_mat.ix[:,:1]

In [ ]:
data_sims.head()

In [ ]:
for i in range(0,len(data_sims.index)):
    for j in range(1,len(data_sims.columns)):
        user = data_sims.index[i]
        product = data_sims.columns[j]
 
        if cust_prod_mat.ix[i][j] == 1:
            data_sims.ix[i][j] = 0
        else:
            product_top_names = data_neighbours.ix[product][1:10]
            product_top_sims = data_ibs.ix[product].sort_values(ascending=False)[1:10]
            user_purchases = data_user.ix[user,product_top_names]
 
            data_sims.ix[i][j] = getScore(user_purchases,product_top_sims)

In [ ]:
data_recommend = pd.DataFrame(index=data_sims.index, columns=['cust_id','Product 1','Product 2','Product 3','Product 4','Product 5','Product 6'])

In [ ]:
data_recommend.ix[0:,0] = data_sims.ix[:,0]

In [ ]:
for i in range(0,len(data_sims.index)):
    data_recommend.ix[i,1:] = data_sims.ix[i,:].sort_values(ascending=False).ix[1:7,].index.transpose()

In [ ]:
data_recommend.head(6)

In [ ]:
len(data_recommend)